In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from statistics import NormalDist
from scipy.stats import norm
import scipy.special as sp

# Downloading the data:

In [2]:
X = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_4\initial_inputs.npy")
y = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_4\initial_outputs.npy")

Here I'm appending the inputs resulted from the acquisition function below, so everytime I get an query, I add it here to the input "x"

In [3]:
X = np.append(X,[[0.22449, 0.326531, 0.346939, 0.387755], [0.413793, 0.37931, 0.344828, 0.413793], [0.344828, 0.37931, 0.37931, 0.413793], 
                [0.344828, 0.37931, 0.586207, 0.448276]], axis=0)
X

array([[0.89698105, 0.72562797, 0.17540431, 0.70169437],
       [0.8893564 , 0.49958786, 0.53926886, 0.50878344],
       [0.25094624, 0.03369313, 0.14538002, 0.49493242],
       [0.34696206, 0.0062504 , 0.76056361, 0.61302356],
       [0.12487118, 0.12977019, 0.38440048, 0.2870761 ],
       [0.80130271, 0.50023109, 0.70664456, 0.19510284],
       [0.24770826, 0.06044543, 0.04218635, 0.44132425],
       [0.74670224, 0.7570915 , 0.36935306, 0.20656628],
       [0.40066503, 0.07257425, 0.88676825, 0.24384229],
       [0.6260706 , 0.58675126, 0.43880578, 0.77885769],
       [0.95713529, 0.59764438, 0.76611385, 0.77620991],
       [0.73281243, 0.14524998, 0.47681272, 0.13336573],
       [0.65511548, 0.07239183, 0.68715175, 0.08151656],
       [0.21973443, 0.83203134, 0.48286416, 0.08256923],
       [0.48859419, 0.2119651 , 0.93917791, 0.37619173],
       [0.16713049, 0.87655456, 0.21723954, 0.95980098],
       [0.21691119, 0.16608583, 0.24137226, 0.77006248],
       [0.38748784, 0.80453226,

Here I'm appending outputs I get from the feedback, so evertime I get the output from the feedback, I add it to the output "y"

In [4]:
y = np.append(y,[-2.70557175, -1.1971128, 0.418945648, -3.469662475], axis=0)

# Fitting in Guassian Distribution

In [5]:
gpr = GaussianProcessRegressor(kernel=None)
gpr.fit(X, y);

In [6]:
x1 = np.linspace(0, 1, 30)

# Creating a grid for grid search

In [7]:
import itertools as it
dim = 4
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)


Implementing the grid search:

In [8]:
X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)

#####################################################################################################
#Below is my previous approach where I used the probability improvement method for the Acquisition Function (you can ignor it)
#ucb = mean + 1.96 * std
#acquisition_function = []
#eta=0.1
#mx = max([-2.70557175, -1.1971128])
#for k in range(len(X_grid)):
 #   acquisition_function.append(1 - NormalDist(mu = mean[k], sigma = std[k]).cdf(mx + eta))
  #  if  mean[k] > mx:
   #     mx = mean[k]
#acquisition_function = np.array(acquisition_function)

# Applying the "Analytic LogEI"

I have found this approach in "Unexpected Improvements to Expected Improvement for Bayesian Optimization" article (expalined in README file)

Now let's define log1mexp which is $$log( 1 - exp(x) )$$

In [9]:
def log1mexp(x):
    return np.log(1-np.exp(x))

$log( 1 - exp(x) )$ is used in the acquistion function later

Recall from README file: 

$$c1 = log(2π) / 2$$
$$c2 =  log(π/2) /2$$
$$erfcx(x) = exp(x^{2})erfc(x)$$
where erfc is the complementary error function.
The "Analytic LogEI" acquistion function is: $$ LogEIy^{∗}(x) = log_h((µ(x) − y^{∗})/σ(x)) + log(σ(x))$$
where $log_h(x)$ is calculated below (based on several conditions mentioned in README file and the article (section 4.1)

eta in the code below is ϵ which is chosen to be 0.1. So Let's apply that below:

In [10]:
z = (mean - max(y)) / std
c1 = np.log(2 * (22/7)) / 2
c2 = np.log((22/7) / 2) / 2
eta=0.1

if (z > -1).any:     # the first condition for log_h 
    log_h = np.log(norm.pdf(z) + z * norm.cdf(z) + 1e-10)   # I added "1e-10" to avoid division by zero

elif (-1/np.sqrt(eta) < z).any() and (z <= -1).any():    # the second condition for log_h 
    log_h = -z**2/(2-c1) + log1mexp(np.log(sp.erfcx(-z/np.sqrt(2)) * abs(z)) + c2)

elif (z <= -1 / np.sqrt(eta)).any():  # the third condition for log_h 
    log_h = -z**2/(2-c1) + 2* np.log(abs(z))

acquisition_function = log_h + np.log(std)
acquisition_function

array([-25.32860737, -25.42064389, -25.51446521, ..., -25.30815818,
       -25.22656512, -25.14834972])

# Obtaining the next Query:

In [11]:
idx_max = np.argmax(acquisition_function)
next_query = X_grid[idx_max]
print(np.round(next_query, decimals=6))

[0.758621 0.       1.       1.      ]
